In [1]:
import itertools
import pandas as pd
import itertools
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [2]:
data = pd.read_csv("data/combined_expression.csv")
data['cluster'].replace([1, 2],[0, 1], inplace=True)
data.head()

,CELL_LINE_NAME,cluster,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,C6orf10,TMEM225,NOTCH4,PBX2,AGER,RNF5,AGPAT1,DFNB59,PRRT1,FKBPL
0,1240123,1,8.319417,3.111183,9.643558,4.757258,3.919757,3.602185,3.329644,9.076950,...,3.085394,3.462811,3.339030,4.614897,3.395845,3.419193,3.971646,3.729310,3.320022,6.447316
1,1240131,0,7.611268,2.704739,10.276079,3.650299,3.481567,3.145538,3.565127,7.861068,...,2.801456,2.985889,3.180068,5.415729,3.299858,3.028414,3.877889,3.911516,3.379405,4.729557
2,1240132,0,7.678658,2.845781,10.180954,3.573048,3.431235,3.090781,4.116643,8.121190,...,2.934962,2.952937,3.164655,5.707506,3.434295,2.961345,4.272194,3.085696,3.002557,5.653588
3,1240134,0,3.265063,3.063746,10.490285,3.340791,3.676912,3.512821,3.873922,8.790851,...,3.041839,3.398847,3.106710,5.773963,3.412641,3.136110,4.422262,3.522122,3.509437,5.953242
4,1240140,0,7.090138,2.988043,10.264692,4.119555,3.432585,3.308033,3.318371,6.927761,...,3.028787,3.225982,3.275820,5.334283,3.864678,3.259242,3.840581,5.809553,3.674587,5.577503


In [3]:
data.shape

(541, 16384)

In [4]:
selected_genes = pd.read_csv('cleaned/boruta.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [5]:
# retrieving proper columns
X = data.loc[:, selected_genes]
y = data['cluster'].values
# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# scaling data
scaler = MinMaxScaler().fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# checking for class proportions
ones, zeroes = 0, 0
for i in y_train:
    if i == 0:
        zeroes += 1
    else:
        ones += 1
print(f'0: {zeroes}, 1: {ones}, total: {len(y_train)}')

0: 289, 1: 143, total: 432


In [7]:
# prevent against class imbalance
sm = SMOTE()
X_train, y_train = sm.fit_sample(X_train, y_train)

In [8]:
# checking for class proportions again
new_ones, new_zeroes = 0, 0
for i in y_train:
    if i == 0:
        new_zeroes += 1
    else:
        new_ones += 1
print(f'0: {new_zeroes}, 1: {new_ones}, total: {len(y_train)}')

0: 289, 1: 289, total: 578


In [9]:
# saving split data for faster future training
pd.DataFrame(X_train).to_csv('split_data/X_train.csv', index=False)
pd.DataFrame(X_test).to_csv('split_data/X_test.csv', index=False)
pd.DataFrame(y_train).to_csv('split_data/y_train.csv', index=False)
pd.DataFrame(y_test).to_csv('split_data/y_test.csv', index=False)